In [1]:
import tweepy
import json
import sys
from kafka import KafkaProducer
from configparser import ConfigParser

In [2]:
import configparser, argparse, logging, socket, tweepy, socket, sys
#from confluent_kafka import Producer
from tweepy.streaming import Stream

In [9]:
print(tweepy.__version__)

4.12.1


In [3]:
config = ConfigParser()
config.read("..\conf\hashtags.conf")

['..\\conf\\hashtags.conf']

In [4]:
consumer_key = config['API_details']['consumer_key']
consumer_secret = config['API_details']['consumer_secret']
access_token = config['API_details']['access_token']
access_secret = config['API_details']['access_secret']
topic = config['Resources']['app_topic_name']

In [10]:
#bootstap_server = config['Kafka_param']['bootstrap.servers']
producer = KafkaProducer(bootstrap_servers=[bootstap_server], api_version=(4,12,1),
                             value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [6]:
location = [float(x) for x in config['API_param']['location'].split(',')]
language = config['API_param']['language'].split(' ')
track_keywords = config['API_param']['track_keywords'].split(' ')

In [11]:
class TwitterStreamListener(tweepy.Stream):

    _kafka_producer = None
    _topic = None
    
    def connect_to_kafka(self):
               
        self._kafka_producer = producer
        self._topic = topic
        
    def on_data(self, data):
        if self._kafka_producer!=None:
            self._kafka_producer.produce(self._topic, value=data)
            self._kafka_producer.flush()
            logging.debug(f"tweet: {data}")
        else:
            print(data)
            self.process_data(data)
            
    def process_data(self, raw_data):
        data = json.loads(raw_data)
        text = data["text"]
        if "extended_tweet" in data:
            text = data["extended_tweet"]["full_text"]
        if '#' in text:
            self._kafka_producer.send(self._topic, value={"text": text})
            self.count += 1
            if self.count % 100 == 0:
                print("Number of tweets sent = ", self.count)

    def on_error(self, status):
        logging.error(status)
        sys.exit(-1)

In [12]:
twitter_conn = TwitterStreamListener(consumer_key, consumer_secret,
                                         access_token, access_secret)

In [13]:
twitter_conn.connect_to_kafka()

Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream encountered HTTP error: 410
HTTP error response text: 
Stream e

In [ ]:
twitter_conn.filter(track=track_keywords)

In [ ]:
twitter_conn.filter(track=track_keywords)